In [ ]:
# default_exp exposure
%load_ext autoreload
%autoreload 2

# exposure
> calculate exposure for a point source from a GTI and FT2 files

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from astropy.io import fits
import numpy as np
import pandas as pd
from scipy.integrate import simps
from light_curves.config import MJD

In [ ]:
#export
def _process_ft2(config, source, ft2_files, gti, effective_area):
    """Process a set of FT2 files, with S/C history data
    Parameters:
        - config -- verbose, cos_theta_max, z_max
        - source -- l,b for position
        - ft2_files -- list of spacecraft files
        - gti -- GTI object with allowed intervals
        - effective_area -- function of energy and angle with respect to zenity
    Generate a dataset with fields:
        - start, stop : start  and stop times in MJD
        - exposure    : calculated exposure using effective area

     """
    # combine the files into a DataFrame with following fields besides START and STOP (lower case for column)
    fields    = ['LIVETIME','RA_SCZ','DEC_SCZ', 'RA_ZENITH','DEC_ZENITH']
    if config.verbose>1:
        print(f'Processing {len(ft2_files)} S/C history (FT2) files')
        print(f'  applying cuts cos(theta) < {config.cos_theta_max},  z < {config.z_max}')
    sc_data=[]
    for filename in ft2_files:
        with fits.open(filename) as hdu:
            scdata = hdu['SC_DATA'].data
            # get times to check against MJD limits and GTI
            start, stop = [MJD(np.array(scdata.START, float)),
                           MJD(np.array(scdata.STOP, float))]
            if config.mjd_range is not None:
                a,b=  config.mjd_range
                if stop[-1]<a:
                    print(f'\tfile {filename}: skip, before selected range' )
                    continue
                elif start[0]>b:
                    print(f'\tfile {filename}: quit, beyond range')
                    break
            # apply GTI to bin center (avoid edge effects?)
            in_gti = gti(0.5*(start+stop))
            if config.verbose>2:
                print(f'\tfile {filename}: {len(start)} entries, {sum(in_gti)} in GTI')
            t = [('start', start[in_gti]), ('stop',stop[in_gti])]+\
                [(field.lower(), np.array(scdata[field][in_gti],np.float32)) for field in fields ]
            sc_data.append( pd.DataFrame(dict(t) ) )
    df = pd.concat(sc_data, ignore_index=True)

    # calculate cosines with respect to sky direction
    sc = source
    ra_r,dec_r = np.radians(sc.ra), np.radians(sc.dec)
    sdec, cdec = np.sin(dec_r), np.cos(dec_r)

    def cosines( ra2, dec2):
        ra2_r =  np.radians(ra2.values)
        dec2_r = np.radians(dec2.values)
        return np.cos(dec2_r)*cdec*np.cos(ra_r-ra2_r) + np.sin(dec2_r)*sdec

    pcosines = cosines(df.ra_scz,    df.dec_scz)
    zcosines = cosines(df.ra_zenith, df.dec_zenith)

    # mask out entries too close to zenith, or too far away from ROI center
    mask =   (pcosines >= config.cos_theta_max) & (zcosines>=np.cos(np.radians(config.z_max)))
    if config.verbose>1:
        print(f'\tFound {len(mask):,} S/C entries:  {sum(mask):,} remain after zenith and theta cuts')
    dfm = df.loc[mask,:]
    livetime = dfm.livetime.values
    config.dfm = dfm ##############debug
    # apply MJD range if present. note times in MJD
    start, stop = dfm.start,dfm.stop
    lims = slice(None)
    if config.mjd_range is not None:
#         a, b = config._get_limits(start)
        a, b = np.searchsorted(start, config.mjd_range)
        if a>0 or b<len(start):
            if config.verbose>1:
                print(f'\tcut from {len(start):,} to {a} - {b}, or {b-a:,} entries after MJD range selection')
            dfm = dfm.iloc[a:b]
            lims = slice(a,b)


    expose = _exposure(config, effective_area, livetime[lims], pcosines[mask][lims])
    return pd.DataFrame(dict(start=start[lims],stop=stop[lims], exposure=expose))

In [ ]:
#export
def _exposure(config, effective_area, livetime, pcosine):
    """return exposure calculated for each pair in livetime and cosines arrays

    uses effective area
    """
    assert len(livetime)==len(pcosine), 'expect equal-length arrays'

    # get a set of energies and associated weights from a trial spectrum

    emin,emax = config.energy_range
    loge1=np.log10(emin); loge2=np.log10(emax)

    edom=np.logspace(loge1, loge2, int((loge2-loge1)*config.bins_per_decade+1))
    if config.verbose>1:
        print(f'Calculate exposure using the energy domain'\
              f' {emin}-{emax} {config.bins_per_decade} bins/decade' )
    base_spectrum = eval(config.base_spectrum) #lambda E: (E/1000)**-2.1
    assert base_spectrum(1000)==1.
    wts = base_spectrum(edom)

    # effectivee area function from
    ea = effective_area

    # a table of the weighted for each pair in livetime and pcosine arrays
    rvals = np.empty([len(wts),len(pcosine)])
    for i,(en,wt) in enumerate(zip(edom,wts)):
        faeff,baeff = ea([en],pcosine)
        rvals[i] = (faeff+baeff)*wt

    aeff = simps(rvals,edom,axis=0)/simps(wts,edom)
    return (aeff*livetime)

In [ ]:
#export
def get_exposure(config, files, gti, source):
    """Return the exposure for the source
    If gti is None, regenerate it
    """
    from  light_curves.load_gti import get_gti
    from light_curves.effective_area import EffectiveArea
    gti = gti or get_gti(config, files.gti)
    aeff = EffectiveArea(file_path = files.aeff)
    return _process_ft2(config, source, files.ft2, gti, aeff)

In [ ]:
from light_curves.config import LCconfig, FileConfiguration, PointSource
# To speed up test, reduce MJD range
config = LCconfig(mjd_range=(54682,55000 ))
files = FileConfiguration()
source = PointSource('Geminga')

exposure = get_exposure(config, files, None, source)

Processing 11 GTI files ...  11 files, 63635 intervals with 3,322 days live time
	cut from 63,635 to 0 - 4816, or 4,816 entries after MJD range selection
	GTI MJD range: 54682.66-55000.03, good fraction 0.82 
Processing 12 S/C history (FT2) files
  applying cuts cos(theta) < 0.4,  z < 100
	file /home/burnett/work/lat-data/ft2/ft2_2008.fits: 362996 entries, 360944 in GTI
	file /home/burnett/work/lat-data/ft2/ft2_2009.fits: 874661 entries, 392949 in GTI
	file /home/burnett/work/lat-data/ft2/ft2_2010.fits: quit, beyond range
	Found 753,893 S/C entries:  235,634 remain after zenith and theta cuts
	cut from 235,634 to 0 - 235575, or 235,575 entries after MJD range selection
Calculate exposure using the energy domain 100.0-1000000.0 4 bins/decade


The result is a tuple of time intervals and the exposure

In [ ]:
exposure

,start,stop,exposure
0,54682.656038,54682.656375,112998.052089
1,54682.656375,54682.656722,116901.160578
2,54682.656722,54682.657069,116857.143435
3,54682.657069,54682.657416,116791.434285
4,54682.657416,54682.657764,116876.550059
...,...,...,...
753747,54999.954488,54999.954835,44173.944047
753748,54999.954835,54999.955183,39003.896685
753749,54999.955183,54999.955530,33775.488160
753818,54999.999546,54999.999893,35121.038690


In [ ]:

# # cache?
# cache_folder = '/tmp/light_curves/'
# os.makedirs(cache_folder, exist_ok=True)
# fn = source.name.replace(' ', '_').replace('+','p')
# filename = os.path.join(cache_folder, fn+'.pkl')
# exposure.to_pickle( fn)
# print(f'Exposure for {source.name}  saved to {filename}')

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_load_gti.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 07_cells.ipynb.
Converted 09_poisson.ipynb.
Converted 10_loglike.ipynb.
Converted 11_lightcurve.ipynb.
Converted index.ipynb.
Sun Dec  6 12:46:05 PST 2020
